### Deploy Master Firebase

In [1]:
#!pip install google-cloud-storage

In [2]:
#!pip install firebase_admin

In [3]:
#!pip install firebase

In [4]:
#!pip install --upgrade pip

In [5]:
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib

In [6]:
#!pip install pyrebase4

In [7]:
import firebase_admin
from firebase_admin import credentials, firestore, storage

In [8]:
#import pyrebase
import pandas as pd
import numpy as np
import os
import glob
from pathlib import Path
import shutil

In [9]:
#import firebase_admin
#from firebase_admin import credentials

cred = credentials.Certificate("/home/kurubal/Documents/Firebase/Flash Card/Api Key/flashcard-1c46e-firebase-adminsdk-b4kn5-5ddbde56b3.json")  # Flash Card
app = firebase_admin.initialize_app(cred)

In [10]:
fire_db = firestore.client()

#### Read Data

In [11]:
data_master = pd.read_excel("/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Arabic/Quran/Deploy Result/Master 17052022.xlsx")
data_master

,surah,ayah,word rank,juz,page,surah arabic,surah latin,word count ayah,letter count ayah,tanzil_clean,...,twogram_bangla,twogram_russian,threegram_string,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
0,1,1,1,1,1,الفاتحة,Al-Fatiha,4,19,بسم,...,আল্লাহর নামে,Во имя Бога,بسم الله الرحمن,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
1,1,1,2,1,1,الفاتحة,Al-Fatiha,4,19,الله,...,পরম করুণাময় ঈশ্বর,Бог Милостивый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,3,1,1,الفاتحة,Al-Fatiha,4,19,الرحمن,...,পরম করুণাময়,Самый Милосердный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,4,1,1,الفاتحة,Al-Fatiha,4,19,الرحيم,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2,1,1,1,الفاتحة,Al-Fatiha,4,18,الحمد,...,সৃষ্টিকর্তাকে ধন্যবাদ,Слава Богу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78242,114,5,4,30,604,الناس,An-Nas,5,20,صدور,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78243,114,5,5,30,604,الناس,An-Nas,5,20,الناس,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78244,114,6,1,30,604,الناس,An-Nas,3,13,من,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78245,114,6,2,30,604,الناس,An-Nas,3,13,الجنة,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_page_surah = data_master.groupby(["page"])["surah latin"].apply(set).reset_index()
df_page_surah

,page,surah latin
0,1,{Al-Fatiha}
1,2,{Al-Baqara}
2,3,{Al-Baqara}
3,4,{Al-Baqara}
4,5,{Al-Baqara}
...,...,...
599,600,"{At-Takathur, Al-Qari'a, Al-'Adiyat}"
600,601,"{Al-Humaza, Al-Feel, Al-'Asr}"
601,602,"{Al-Kawthar, Quraysh, Al-Ma'oon}"
602,603,"{Al-Kafiroon, Al-Masad, An-Nasr}"


In [13]:
df_page_surah_ayah = data_master.groupby(["page","surah latin"])["ayah"].apply(set).reset_index()
df_page_surah_ayah

,page,surah latin,ayah
0,1,Al-Fatiha,"{1, 2, 3, 4, 5, 6, 7}"
1,2,Al-Baqara,"{0, 1, 2, 3, 4, 5}"
2,3,Al-Baqara,"{6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16}"
3,4,Al-Baqara,"{17, 18, 19, 20, 21, 22, 23, 24}"
4,5,Al-Baqara,"{25, 26, 27, 28, 29}"
...,...,...,...
657,603,Al-Masad,"{0, 1, 2, 3, 4, 5}"
658,603,An-Nasr,"{0, 1, 2, 3}"
659,604,Al-Falaq,"{0, 1, 2, 3, 4, 5}"
660,604,Al-Ikhlas,"{0, 1, 2, 3, 4}"


#### Sending Data To Firestore

##### Page Data

In [14]:
data_master_select = data_master.drop(["word count ayah","letter count ayah","buckwalter_word_tanzil_clean","buckwalter_word_tanzil_plain","root","tag","2gram","3gram","index"], axis=1)
data_master_select

,surah,ayah,word rank,juz,page,surah arabic,surah latin,tanzil_clean,tanzil_plain,root_arabic,...,twogram_bangla,twogram_russian,threegram_string,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
0,1,1,1,1,1,الفاتحة,Al-Fatiha,بسم,بِسْمِ,سمو,...,আল্লাহর নামে,Во имя Бога,بسم الله الرحمن,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
1,1,1,2,1,1,الفاتحة,Al-Fatiha,الله,اللَّهِ,اله,...,পরম করুণাময় ঈশ্বর,Бог Милостивый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,1,3,1,1,الفاتحة,Al-Fatiha,الرحمن,الرَّحْمَٰنِ,رحم,...,পরম করুণাময়,Самый Милосердный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1,1,4,1,1,الفاتحة,Al-Fatiha,الرحيم,الرَّحِيمِ,رحم,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1,2,1,1,1,الفاتحة,Al-Fatiha,الحمد,الْحَمْدُ,حمد,...,সৃষ্টিকর্তাকে ধন্যবাদ,Слава Богу,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78242,114,5,4,30,604,الناس,An-Nas,صدور,صُدُورِ,صدر,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78243,114,5,5,30,604,الناس,An-Nas,الناس,النَّاسِ,نوس,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78244,114,6,1,30,604,الناس,An-Nas,من,مِنَ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78245,114,6,2,30,604,الناس,An-Nas,الجنة,الْجِنَّةِ,جنن,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
data_master_select.columns

Index(['surah', 'ayah', 'word rank', 'juz', 'page', 'surah arabic',
       'surah latin', 'tanzil_clean', 'tanzil_plain', 'root_arabic', 'level 1',
       'level 2', 'level 3', 'level 4', 'start_time', 'end_time',
       'repeat flag', 'arabic', 'translate_english', 'translate_urdu',
       'translate_hindi', 'translate_indonesian', 'translate_bangla',
       'translate_turkish', 'translate_russian', 'twogram_string',
       'twogram_english', 'twogram_turkish', 'twogram_urdu', 'twogram_hindi',
       'twogram_indonesian', 'twogram_bangla', 'twogram_russian',
       'threegram_string', 'threegram_english', 'threegram_turkish',
       'threegram_urdu', 'threegram_hindi', 'threegram_indonesian',
       'threegram_bangla', 'threegram_russian'],
      dtype='object')

In [18]:
data_master_select[data_master_select["page"] == 604]

,surah,ayah,word rank,juz,page,surah arabic,surah latin,tanzil_clean,tanzil_plain,root_arabic,...,twogram_bangla,twogram_russian,threegram_string,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
78177,112,0,1,30,604,الإخلاص,Al-Ikhlas,بسم,بِسْمِ,NaN,...,আল্লাহর নামে,Во имя Бога,بسم الله الرحمن,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
78178,112,0,2,30,604,الإخلاص,Al-Ikhlas,الله,اللَّهِ,NaN,...,পরম করুণাময় ঈশ্বর,Бог Милостивый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78179,112,0,3,30,604,الإخلاص,Al-Ikhlas,الرحمن,الرَّحْمَٰنِ,NaN,...,পরম করুণাময়,Самый Милосердный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78180,112,0,4,30,604,الإخلاص,Al-Ikhlas,الرحيم,الرَّحِيمِ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78181,112,1,1,30,604,الإخلاص,Al-Ikhlas,قل,قُلْ,قول,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78242,114,5,4,30,604,الناس,An-Nas,صدور,صُدُورِ,صدر,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78243,114,5,5,30,604,الناس,An-Nas,الناس,النَّاسِ,نوس,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78244,114,6,1,30,604,الناس,An-Nas,من,مِنَ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78245,114,6,2,30,604,الناس,An-Nas,الجنة,الْجِنَّةِ,جنن,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
for page_num in range(1,605):
    df_var = data_master_select[data_master_select["page"] == page_num]

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277


In [16]:
data_master_select[((data_master_select["page"] == 604) & (data_master_select["surah latin"] == "An-Nas") & (data_master_select["ayah"] == 0))]

,surah,ayah,word rank,juz,page,surah arabic,surah latin,tanzil_clean,tanzil_plain,root_arabic,...,twogram_bangla,twogram_russian,threegram_string,threegram_english,threegram_turkish,threegram_urdu,threegram_hindi,threegram_indonesian,threegram_bangla,threegram_russian
78223,114,0,1,30,604,الناس,An-Nas,بسم,بِسْمِ,NaN,...,আল্লাহর নামে,Во имя Бога,بسم الله الرحمن,"in the name of ALLAH, the most mercifull",merhametli olan Allah adıyla,اللہ کے نام سے جو سب سے زیادہ رحم کرنے والا ہے۔,"अल्लाह के नाम पर, सबसे दयालु",dengan menyebut nama ALLAH yang maha penyayang,পরম করুণাময় আল্লাহর নামে,во имя АЛЛАХА Милостивого
78224,114,0,2,30,604,الناس,An-Nas,الله,اللَّهِ,NaN,...,পরম করুণাময় ঈশ্বর,Бог Милостивый,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78225,114,0,3,30,604,الناس,An-Nas,الرحمن,الرَّحْمَٰنِ,NaN,...,পরম করুণাময়,Самый Милосердный,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78226,114,0,4,30,604,الناس,An-Nas,الرحيم,الرَّحِيمِ,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
for i in range(len(df_page_surah)):
    page = df_page_surah.loc[i,"page"]
    surah_latin = df_page_surah.loc[i,"surah latin"]
    ayah = df_page_surah.loc[i,"ayah"]
    df_var = data_master_select[(data_master_select["page"] == page) & (data_master_select["surah latin"] == surah_latin)]
    for ayah_num in ayah:
        df_var2 = df_var[df_var["ayah"] == ayah_num]

In [23]:
fire_db.collection("pages").document(f"page{page}").set(data)

In [24]:
#fire_db.collection("Root_Data").document(f"root{sequence}").collection(f"{root}").document(f"{arabic_simple}").set(data)
#fire_db.collection("Root_Data").document(f"root{sequence}").set(data)

# Not: Eger document ile bitiyorsa data set ile gönderiliyor.
#fire_db.collection("Root_Data").document(f"root{sequence}").collection("Detail").add(data)  
 
# Not: Eğer collection ile bitiyorsa data add ile gönderiliyor ve collection dan sonraki document adları rastgele oluşturuluyor.
#fire_db.collection(u'cities').document(u'DC').delete()                                                                                             